<a href="https://colab.research.google.com/github/wendikardian/exercise_machine_learning/blob/main/TFDS_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import pandas as pd
import numpy as np

import tensorflow as tf

from tensorflow import feature_column
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

In [10]:
url= 'https://raw.githubusercontent.com/natashayulian/diamond_dataset/master/diamonds.csv'
df = pd.read_csv(url)
df

,Unnamed: 0,carat,cut,color,clarity,depth,table,price,x,y,z
0,1,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,2,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
2,3,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
3,4,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
4,5,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75
...,...,...,...,...,...,...,...,...,...,...,...
53935,53936,0.72,Ideal,D,SI1,60.8,57.0,2757,5.75,5.76,3.50
53936,53937,0.72,Good,D,SI1,63.1,55.0,2757,5.69,5.75,3.61
53937,53938,0.70,Very Good,D,SI1,62.8,60.0,2757,5.66,5.68,3.56
53938,53939,0.86,Premium,H,SI2,61.0,58.0,2757,6.15,6.12,3.74


In [11]:
train, test = train_test_split(df, test_size=0.2)
train, val = train_test_split(train, test_size=0.2)
print(len(train), 'train examples')
print(len(val), 'validation examples')
print(len(test), 'test examples')

34521 train examples
8631 validation examples
10788 test examples


In [12]:
#Membuat kolom target
# 0 = low; 1 = high
df['target'] = np.where(df['price'] <= 1000, 0, 1)
# Drop un-used columns.
df = df.drop(columns=['price'])


In [13]:
df

,Unnamed: 0,carat,cut,color,clarity,depth,table,x,y,z,target
0,1,0.23,Ideal,E,SI2,61.5,55.0,3.95,3.98,2.43,0
1,2,0.21,Premium,E,SI1,59.8,61.0,3.89,3.84,2.31,0
2,3,0.23,Good,E,VS1,56.9,65.0,4.05,4.07,2.31,0
3,4,0.29,Premium,I,VS2,62.4,58.0,4.20,4.23,2.63,0
4,5,0.31,Good,J,SI2,63.3,58.0,4.34,4.35,2.75,0
...,...,...,...,...,...,...,...,...,...,...,...
53935,53936,0.72,Ideal,D,SI1,60.8,57.0,5.75,5.76,3.50,1
53936,53937,0.72,Good,D,SI1,63.1,55.0,5.69,5.75,3.61,1
53937,53938,0.70,Very Good,D,SI1,62.8,60.0,5.66,5.68,3.56,1
53938,53939,0.86,Premium,H,SI2,61.0,58.0,6.15,6.12,3.74,1


In [14]:
def df_to_dataset(dataframe, shuffle=True, batch_size=32):
  dataframe = df.copy()
  labels = dataframe.pop('target')
  ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
  if shuffle:
    ds = ds.shuffle(buffer_size=len(dataframe))
  ds = ds.batch(batch_size)
  return ds


batch_size = 10
train_ds = df_to_dataset(train, batch_size = batch_size)
val_ds = df_to_dataset(val, shuffle=False, batch_size = batch_size)
test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)

In [15]:
#Pilih feature column mana yang akan digunakan
feature_columns = []
# numeric column
for header in ['carat', 'depth', 'x', 'y', 'z']:
  feature_columns.append(feature_column.numeric_column(header))

#membuat feature layer
feature_layer = tf.keras.layers.DenseFeatures(feature_columns)
batch_size = 32
train_ds = df_to_dataset(train, batch_size=batch_size)
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)


Instructions for updating:
Use Keras preprocessing layers instead, either directly or via the `tf.keras.utils.FeatureSpace` utility. Each of `tf.feature_column.*` has a functional equivalent in `tf.keras.layers` for feature preprocessing when training a Keras model.


In [16]:
model = tf.keras.Sequential([
  feature_layer,
  layers.Dense(128, activation='relu'),
  layers.Dense(128, activation='relu'),
  layers.Dropout(.1),
  layers.Dense(1)
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.fit(train_ds,
          validation_data=val_ds,
          epochs=10)

Epoch 1/10
1686/1686 [==============================] - 12s 6ms/step - loss: 0.2093 - accuracy: 0.9079 - val_loss: 0.1453 - val_accuracy: 0.9374
Epoch 2/10
1686/1686 [==============================] - 8s 4ms/step - loss: 0.1595 - accuracy: 0.9331 - val_loss: 0.1874 - val_accuracy: 0.9282
Epoch 3/10
1686/1686 [==============================] - 9s 5ms/step - loss: 0.1550 - accuracy: 0.9347 - val_loss: 0.1483 - val_accuracy: 0.9352
Epoch 4/10
1686/1686 [==============================] - 11s 6ms/step - loss: 0.1548 - accuracy: 0.9352 - val_loss: 0.1588 - val_accuracy: 0.9317
Epoch 5/10
1686/1686 [==============================] - 7s 4ms/step - loss: 0.1519 - accuracy: 0.9356 - val_loss: 0.1500 - val_accuracy: 0.9356
Epoch 6/10
1686/1686 [==============================] - 8s 4ms/step - loss: 0.1490 - accuracy: 0.9356 - val_loss: 0.1411 - val_accuracy: 0.9382
Epoch 7/10
1686/1686 [==============================] - 9s 5ms/step - loss: 0.1475 - accuracy: 0.9374 - val_loss: 0.1648 - val_accurac